# Хеш-функции: от свойств до реализации

Этот notebook охватывает:
- **Часть 1:** Хеш-функции — свойства и лавинный эффект (CRYPTO-03)
- **Часть 2:** SHA-256 с нуля на чистом Python (CRYPTO-04)
- **Часть 3:** Keccak-256 vs SHA-3-256 (CRYPTO-05)
- **Упражнения и челлендж**

---
## Часть 1: Хеш-функции (CRYPTO-03)

### 1.1 Базовое использование hashlib

In [ ]:
import hashlib

# SHA-256
h = hashlib.sha256(b"Hello, blockchain!").hexdigest()
print(f"SHA-256:   {h}")
print(f"Длина:     {len(h)} hex символов = {len(h)*4} бит")

# SHA-3-256
h3 = hashlib.sha3_256(b"Hello, blockchain!").hexdigest()
print(f"SHA-3-256: {h3}")

# BLAKE2b (256 бит)
hb = hashlib.blake2b(b"Hello, blockchain!", digest_size=32).hexdigest()
print(f"BLAKE2b:   {hb}")

### 1.2 Лавинный эффект

Изменение одного бита во входе должно менять ~50% бит выхода.

In [ ]:
def count_diff_bits(hex1: str, hex2: str) -> int:
    """Подсчет отличающихся бит между двумя hex-строками."""
    val1 = int(hex1, 16)
    val2 = int(hex2, 16)
    xor = val1 ^ val2
    return bin(xor).count('1')

# Лавинный эффект: "Hello" vs "hello" (1 бит разницы в ASCII)
h1 = hashlib.sha256(b"Hello").hexdigest()
h2 = hashlib.sha256(b"hello").hexdigest()

diff = count_diff_bits(h1, h2)
print(f'SHA-256("Hello"): {h1}')
print(f'SHA-256("hello"): {h2}')
print(f"Отличающихся бит: {diff} / 256 ({diff/256*100:.1f}%)")

In [ ]:
import os

# Статистика лавинного эффекта на 1000 случайных входах
diffs = []
for _ in range(1000):
    data = os.urandom(32)
    # Инвертируем один бит
    data_modified = bytearray(data)
    data_modified[0] ^= 0x01  # Инвертируем младший бит первого байта
    
    h_orig = hashlib.sha256(data).hexdigest()
    h_mod = hashlib.sha256(bytes(data_modified)).hexdigest()
    diffs.append(count_diff_bits(h_orig, h_mod))

avg_diff = sum(diffs) / len(diffs)
min_diff = min(diffs)
max_diff = max(diffs)

print(f"Статистика лавинного эффекта (1000 тестов):")
print(f"  Среднее: {avg_diff:.1f} / 256 ({avg_diff/256*100:.1f}%)")
print(f"  Мин:     {min_diff} / 256")
print(f"  Макс:    {max_diff} / 256")
print(f"  Ожидание: ~128 / 256 (50%)")

### 1.3 Парадокс дней рождения (симуляция)

Для хеш-функции с выходом `n` бит, коллизия ожидается после ~`2^(n/2)` хешей.
Симулируем с маленьким пространством (16 бит = 65536 возможных хешей).

In [ ]:
import math

def birthday_simulation(hash_bits: int, trials: int = 100) -> float:
    """Симуляция парадокса дней рождения.
    
    Возвращает среднее количество хешей до первой коллизии
    для усеченного хеша длиной hash_bits бит.
    """
    mask = (1 << hash_bits) - 1
    total_hashes = 0
    
    for _ in range(trials):
        seen = set()
        count = 0
        while True:
            data = os.urandom(16)
            h = int(hashlib.sha256(data).hexdigest(), 16) & mask
            count += 1
            if h in seen:
                break
            seen.add(h)
        total_hashes += count
    
    return total_hashes / trials

# Симуляция для разных размеров хеша
for bits in [8, 12, 16, 20]:
    avg = birthday_simulation(bits, trials=50)
    expected = math.sqrt(math.pi / 2 * (2 ** bits))
    print(f"{bits}-bit hash: коллизия после ~{avg:.0f} хешей "
          f"(теоретически: ~{expected:.0f}, 2^{bits/2:.0f} = {2**(bits//2)})")

---
## Часть 2: SHA-256 с нуля (CRYPTO-04)

Реализация по спецификации FIPS PUB 180-4.

### 2.1 Константы

In [ ]:
# Модуль 2^32 для 32-битной арифметики
MOD = 0x100000000  # 2^32

# Начальные хеш-значения (первые 32 бита дробных частей sqrt первых 8 простых)
INITIAL_HASH_VALUES = [
    0x6a09e667,  # sqrt(2)
    0xbb67ae85,  # sqrt(3)
    0x3c6ef372,  # sqrt(5)
    0xa54ff53a,  # sqrt(7)
    0x510e527f,  # sqrt(11)
    0x9b05688c,  # sqrt(13)
    0x1f83d9ab,  # sqrt(17)
    0x5be0cd19,  # sqrt(19)
]

# Раундовые константы (первые 32 бита дробных частей cbrt первых 64 простых)
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2,
]

print(f"Начальных значений: {len(INITIAL_HASH_VALUES)}")
print(f"Раундовых констант: {len(K)}")
print(f"K[0] = 0x{K[0]:08x} (проверка: 0x428a2f98)")

### 2.2 Битовые операции

In [ ]:
def rotr(x: int, n: int) -> int:
    """Циклический сдвиг вправо (32 бита)."""
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def shr(x: int, n: int) -> int:
    """Логический сдвиг вправо."""
    return x >> n

# SHA-256 логические функции
def Ch(e: int, f: int, g: int) -> int:
    """Choice: e выбирает между f (бит=1) и g (бит=0)."""
    return (e & f) ^ (~e & g) & 0xFFFFFFFF

def Maj(a: int, b: int, c: int) -> int:
    """Majority: результат = бит большинства из a, b, c."""
    return (a & b) ^ (a & c) ^ (b & c)

def Sigma0(a: int) -> int:
    """Большая Sigma0 (для переменной a)."""
    return rotr(a, 2) ^ rotr(a, 13) ^ rotr(a, 22)

def Sigma1(e: int) -> int:
    """Большая Sigma1 (для переменной e)."""
    return rotr(e, 6) ^ rotr(e, 11) ^ rotr(e, 25)

def sigma0(x: int) -> int:
    """Малая sigma0 (для расписания сообщений)."""
    return rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)

def sigma1(x: int) -> int:
    """Малая sigma1 (для расписания сообщений)."""
    return rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10)

# Проверка
print(f"rotr(0x428a2f98, 7) = 0x{rotr(0x428a2f98, 7):08x}")
print(f"Ch(0xff, 0xaa, 0x55)  = 0x{Ch(0xff, 0xaa, 0x55):08x} (ожидание: 0xaa)")
print(f"Maj(0xff, 0xaa, 0x55) = 0x{Maj(0xff, 0xaa, 0x55):08x} (ожидание: 0xff)")

### 2.3 Дополнение сообщения (Padding)

In [ ]:
def pad_message(msg: bytes) -> bytes:
    """Дополняет сообщение до длины, кратной 512 битам (64 байтам).
    
    1. Добавляем байт 0x80 (бит '1' + 7 нулевых бит)
    2. Добавляем нулевые байты до тех пор, пока длина != 56 mod 64
    3. Добавляем исходную длину в битах как 8 байт big-endian
    """
    msg_len_bits = len(msg) * 8
    
    # Шаг 1: добавляем 0x80
    msg += b'\x80'
    
    # Шаг 2: добавляем нули
    while len(msg) % 64 != 56:
        msg += b'\x00'
    
    # Шаг 3: добавляем длину (64-bit big-endian)
    msg += msg_len_bits.to_bytes(8, byteorder='big')
    
    return msg

# Тест
padded = pad_message(b'abc')
print(f"Исходное: b'abc' ({3} байта)")
print(f"Дополненное: {len(padded)} байт ({len(padded)*8} бит)")
print(f"Hex: {padded.hex()}")
print(f"Количество блоков: {len(padded) // 64}")

### 2.4 Разбиение на блоки и расписание сообщений

In [ ]:
import struct

def parse_blocks(padded: bytes) -> list:
    """Разбивает дополненное сообщение на блоки по 512 бит (64 байта).
    
    Каждый блок — это список из 16 слов (32 бита каждое).
    """
    blocks = []
    for i in range(0, len(padded), 64):
        block = padded[i:i+64]
        # Разбиваем блок на 16 слов по 4 байта (big-endian)
        words = list(struct.unpack('>16I', block))
        blocks.append(words)
    return blocks

def message_schedule(block: list) -> list:
    """Расширяет 16 слов блока до 64 слов (расписание сообщений).
    
    W[0..15] = слова из блока
    W[t] = sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]  для t = 16..63
    """
    W = list(block)  # W[0..15]
    for t in range(16, 64):
        w = (sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]) % MOD
        W.append(w)
    return W

# Тест
blocks = parse_blocks(padded)
print(f"Количество блоков: {len(blocks)}")
print(f"Первые 4 слова блока 0:")
for i in range(4):
    print(f"  W[{i}] = 0x{blocks[0][i]:08x}")

W = message_schedule(blocks[0])
print(f"\nРасписание сообщений (64 слова):")
print(f"  W[16] = 0x{W[16]:08x}")
print(f"  W[63] = 0x{W[63]:08x}")

### 2.5 Полная реализация SHA-256

In [ ]:
def sha256_educational(message: bytes, verbose: bool = False) -> str:
    """Полная реализация SHA-256 по FIPS PUB 180-4.
    
    Args:
        message: Входное сообщение (байты)
        verbose: Если True, выводит значения переменных на каждом раунде
    
    Returns:
        Hex-строка хеша (64 символа = 256 бит)
    """
    # Шаг 1: Дополнение
    padded = pad_message(bytearray(message))  # bytearray для мутабельности
    
    # Шаг 2: Разбиение на блоки
    blocks = parse_blocks(padded)
    
    # Шаг 3: Инициализация хеш-значений
    H = list(INITIAL_HASH_VALUES)
    
    # Шаг 4: Обработка каждого блока
    for block_idx, block in enumerate(blocks):
        if verbose:
            print(f"\n{'='*60}")
            print(f"Блок {block_idx + 1} / {len(blocks)}")
            print(f"{'='*60}")
        
        # Расписание сообщений
        W = message_schedule(block)
        
        # Инициализация рабочих переменных
        a, b, c, d, e, f, g, h = H
        
        if verbose:
            print(f"Начальное состояние:")
            print(f"  a={a:08x} b={b:08x} c={c:08x} d={d:08x}")
            print(f"  e={e:08x} f={f:08x} g={g:08x} h={h:08x}")
        
        # 64 раунда сжатия
        for i in range(64):
            T1 = (h + Sigma1(e) + Ch(e, f, g) + K[i] + W[i]) % MOD
            T2 = (Sigma0(a) + Maj(a, b, c)) % MOD
            
            h = g
            g = f
            f = e
            e = (d + T1) % MOD
            d = c
            c = b
            b = a
            a = (T1 + T2) % MOD
            
            if verbose:
                print(f"Раунд {i:2d}: a={a:08x} b={b:08x} c={c:08x} d={d:08x}"
                      f" e={e:08x} f={f:08x} g={g:08x} h={h:08x}")
        
        # Прибавляем к текущему хешу
        H[0] = (H[0] + a) % MOD
        H[1] = (H[1] + b) % MOD
        H[2] = (H[2] + c) % MOD
        H[3] = (H[3] + d) % MOD
        H[4] = (H[4] + e) % MOD
        H[5] = (H[5] + f) % MOD
        H[6] = (H[6] + g) % MOD
        H[7] = (H[7] + h) % MOD
        
        if verbose:
            print(f"\nХеш после блока {block_idx + 1}:")
            print(f"  {''.join(f'{x:08x}' for x in H)}")
    
    # Финальный хеш
    return ''.join(f'{x:08x}' for x in H)

print("SHA-256 реализация загружена!")

### 2.6 Проверка: наша реализация vs hashlib

In [ ]:
# Тестовый вектор 1: "abc" (из FIPS 180-4)
our_hash = sha256_educational(b"abc")
lib_hash = hashlib.sha256(b"abc").hexdigest()

print(f"Наша:    {our_hash}")
print(f"hashlib: {lib_hash}")
assert our_hash == lib_hash, "ОШИБКА: хеши не совпадают!"
print("Тест 1 пройден: 'abc'")

# Тестовый вектор 2: пустая строка
our_hash2 = sha256_educational(b"")
lib_hash2 = hashlib.sha256(b"").hexdigest()
assert our_hash2 == lib_hash2, "ОШИБКА: хеши не совпадают!"
print(f"Тест 2 пройден: '' -> {our_hash2[:16]}...")

# Тестовый вектор 3: длинная строка (2 блока)
long_msg = b"abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq"
our_hash3 = sha256_educational(long_msg)
lib_hash3 = hashlib.sha256(long_msg).hexdigest()
assert our_hash3 == lib_hash3, "ОШИБКА: хеши не совпадают!"
print(f"Тест 3 пройден: длинная строка (2 блока) -> {our_hash3[:16]}...")

# Тестовый вектор 4: случайные данные
random_data = os.urandom(100)
our_hash4 = sha256_educational(random_data)
lib_hash4 = hashlib.sha256(random_data).hexdigest()
assert our_hash4 == lib_hash4, "ОШИБКА: хеши не совпадают!"
print(f"Тест 4 пройден: 100 случайных байт -> {our_hash4[:16]}...")

print("\nВсе тесты пройдены! Наша реализация SHA-256 корректна.")

### 2.7 Пошаговый вывод (verbose mode)

In [ ]:
# Показываем первые 5 и последние 2 раунда для "abc"
print("SHA-256 раунды для 'abc':")
sha256_educational(b"abc", verbose=True)

---
## Часть 3: Keccak-256 vs SHA-3-256 (CRYPTO-05)

### 3.1 Доказательство разницы

In [ ]:
from Crypto.Hash import keccak

message = b"hello"

# SHA-3-256 (стандарт NIST, FIPS 202)
sha3_hash = hashlib.sha3_256(message).hexdigest()

# Keccak-256 (Ethereum)
keccak_hash = keccak.new(data=message, digest_bits=256).hexdigest()

print(f"Сообщение:  {message}")
print(f"SHA-3-256:  {sha3_hash}")
print(f"Keccak-256: {keccak_hash}")
print(f"\nСовпадают?  {sha3_hash == keccak_hash}")
print(f"\nОтличающихся бит: {count_diff_bits(sha3_hash, keccak_hash)} / 256")

print("\nВАЖНО: Keccak-256 и SHA-3-256 — это РАЗНЫЕ алгоритмы!")
print("Ethereum использует Keccak-256, а НЕ SHA-3-256.")

### 3.2 Вычисление Ethereum-адреса из публичного ключа

In [ ]:
def eth_address_from_pubkey(public_key_hex: str) -> str:
    """Вычисляет Ethereum-адрес из публичного ключа.
    
    Args:
        public_key_hex: 128 hex символов (64 байта) — без префикса 0x04
    
    Returns:
        Ethereum-адрес в формате 0x...
    """
    pubkey_bytes = bytes.fromhex(public_key_hex)
    assert len(pubkey_bytes) == 64, f"Ожидается 64 байта, получено {len(pubkey_bytes)}"
    
    # Keccak-256
    k = keccak.new(data=pubkey_bytes, digest_bits=256)
    hash_hex = k.hexdigest()
    
    # Последние 20 байт (40 hex символов)
    address = "0x" + hash_hex[-40:]
    return address

# Пример: известная пара ключ-адрес
# Vitalik Buterin public key (пример — NOT real)
example_pubkey = (
    "3a443d8381a6798a70c6ff9304bdc8cb0163c23211d11628fae52ef9e0dca11a"
    "001cf066d56a8156fc201cd5df8a36ef694eecd258903fca7086c1fae7441e1d"
)

addr = eth_address_from_pubkey(example_pubkey)
print(f"Публичный ключ: {example_pubkey[:32]}...")
print(f"Ethereum адрес: {addr}")

### 3.3 Идентификатор функции Solidity

In [ ]:
def function_selector(signature: str) -> str:
    """Вычисляет 4-байтовый селектор функции Solidity."""
    k = keccak.new(data=signature.encode(), digest_bits=256)
    return "0x" + k.hexdigest()[:8]

# Популярные функции ERC-20
functions = [
    "transfer(address,uint256)",
    "approve(address,uint256)",
    "transferFrom(address,address,uint256)",
    "balanceOf(address)",
    "totalSupply()",
]

print("Селекторы функций ERC-20:")
print("-" * 60)
for func in functions:
    sel = function_selector(func)
    print(f"{sel}  {func}")

---
## Упражнения

### Упражнение 1: Реализуйте SHA-256 без подглядывания

Попробуйте написать SHA-256 самостоятельно, используя только спецификацию FIPS 180-4.
Подсказки:
1. Начните с padding
2. Реализуйте bitwise функции (rotr, Ch, Maj, Sigma0, Sigma1, sigma0, sigma1)
3. Реализуйте message schedule
4. Реализуйте 64 раунда
5. Проверьте: `assert your_sha256(b'abc') == hashlib.sha256(b'abc').hexdigest()`

In [ ]:
# Ваша реализация здесь
def my_sha256(message: bytes) -> str:
    """Ваша реализация SHA-256."""
    # TODO: реализуйте самостоятельно
    pass

# Проверка
# assert my_sha256(b"abc") == hashlib.sha256(b"abc").hexdigest()

### Упражнение 2: Статистика лавинного эффекта

Измерьте лавинный эффект для 1000 случайных входов. Постройте гистограмму.

In [ ]:
# Подсказка: используйте count_diff_bits из Части 1
# TODO: сгенерируйте 1000 случайных 32-байтных сообщений
# TODO: для каждого — инвертируйте 1 бит и сравните хеши
# TODO: выведите статистику (среднее, мин, макс, стандартное отклонение)

### Упражнение 3: Мини Proof-of-Work

Найдите nonce (число), при котором SHA-256 от строки `"block_data_" + str(nonce)` начинается с заданного количества нулей.

In [ ]:
import time

def mini_pow(data: str, difficulty: int) -> tuple:
    """Находит nonce, при котором хеш начинается с difficulty нулей.
    
    Returns:
        (nonce, hash_hex, elapsed_time)
    """
    target_prefix = "0" * difficulty
    nonce = 0
    start = time.time()
    
    while True:
        message = f"{data}{nonce}".encode()
        h = hashlib.sha256(message).hexdigest()
        if h.startswith(target_prefix):
            elapsed = time.time() - start
            return nonce, h, elapsed
        nonce += 1

# Попробуйте с разной сложностью
for diff in [1, 2, 3, 4, 5]:
    nonce, h, t = mini_pow("block_data_", diff)
    print(f"Сложность {diff}: nonce={nonce:>8d}, хеш={h[:20]}..., время={t:.3f}с")

---
## Челлендж: Length Extension Attack

SHA-256 (Merkle-Damgard) уязвима к length extension attack.
Зная `H(m)` и длину `m`, можно вычислить `H(m || padding || m')` без знания `m`.

Реализуйте эту атаку и покажите, почему Keccak (конструкция губки) к ней невосприимчива.

In [ ]:
# Подсказка:
# 1. Возьмите хеш H(secret || message), где secret неизвестен
# 2. Используйте H как начальное состояние (вместо INITIAL_HASH_VALUES)
# 3. Вычислите padding для secret || message (зная только длину secret)
# 4. Допишите дополнительные данные и продолжите хеширование
# 5. Проверьте, что результат совпадает с H(secret || message || padding || extra)

def length_extension_attack(
    known_hash: str,       # H(secret || message)
    known_message: bytes,   # message (без secret)
    secret_length: int,     # длина secret в байтах
    extra_data: bytes       # данные для дописывания
) -> tuple:
    """Выполняет length extension attack на SHA-256.
    
    Returns:
        (extended_hash, forged_message_suffix)
    """
    # TODO: реализуйте атаку
    pass

# Проверка:
# secret = b"my_secret_key"
# message = b"amount=100"
# legit_hash = hashlib.sha256(secret + message).hexdigest()
# 
# extended_hash, suffix = length_extension_attack(
#     legit_hash, message, len(secret), b"&amount=1000000"
# )
# 
# forged = secret + message + suffix
# assert hashlib.sha256(forged).hexdigest() == extended_hash